In [1]:
"""
download original bert code
"""

!git clone https://github.com/google-research/bert.git

Cloning into 'bert'...
remote: Enumerating objects: 333, done.
remote: Total 333 (delta 0), reused 0 (delta 0), pack-reused 333
Receiving objects: 100% (333/333), 282.45 KiB | 30.00 KiB/s, done.
Resolving deltas: 100% (183/183), done.


In [4]:
"""
download chinese bert pretrained weights
"""

!wget https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip

--2019-09-28 09:12:35--  https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.24.16, 2404:6800:4008:803::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.24.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 381892918 (364M) [application/zip]
Saving to: ‘chinese_L-12_H-768_A-12.zip’

chinese_L-12_H-768_ 100%[===================>] 364.20M  8.78MB/s    in 54s     

2019-09-28 09:13:30 (6.74 MB/s) - ‘chinese_L-12_H-768_A-12.zip’ saved [381892918/381892918]



In [5]:
!ls

 1.h5
 bert
 bert_text_gene.ipynb
 chinese_L-12_H-768_A-12.zip
 embedding-lstm.py
'Fan Ren Xiu Xian Chuan [ Geng Xin Zhi  Di - Wang Yu.txt'
 one_hot_lstm.py
 params.pkl
 README.md
 text_generation.py


In [6]:
!unzip chinese_L-12_H-768_A-12.zip

Archive:  chinese_L-12_H-768_A-12.zip
   creating: chinese_L-12_H-768_A-12/
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: chinese_L-12_H-768_A-12/vocab.txt  
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: chinese_L-12_H-768_A-12/bert_config.json  


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.listdir("drive/My Drive")

['Colab Notebooks',
 '字向量训练数据',
 'colab_dir',
 'ner_ckpt4',
 'ner_ckpt5',
 'Untitled spreadsheet.gsheet']

In [1]:
import collections
import csv
import os
import re
import time
import random
import tensorflow as tf
from tqdm import tqdm
import numpy as np

import bert.modeling as modeling
import bert.optimization as optimization
import bert.tokenization as tokenization


In [2]:
"""
some params

"""

class FLAGS:
    data_path = "Fan Ren Xiu Xian Chuan [ Geng Xin Zhi  Di - Wang Yu.txt"
    bert_config_file = 'chinese_L-12_H-768_A-12/bert_config.json'
    init_checkpoint = 'chinese_L-12_H-768_A-12/bert_model.ckpt'
    vocab_file = 'chinese_L-12_H-768_A-12/vocab.txt'
    ckpt_weights = 'drive/My Drive/gene_ckpt'
    batch_min_length = 20
    batch_max_length = 50
    do_lower_case = True
    is_training = True
    train_test_split = 0.8
    num_train_epochs = 10
    train_batch_size = 32
    test_batch_size = 64
    learning_rate = 5e-5
    warmup_proportion = 0.1

In [7]:
"""
data preprocessing
"""

with open(FLAGS.data_path, encoding='utf-8') as f:
    text = f.read()

text = re.sub("------章节内容开始-------", "", text)
text = re.sub("[(（].{2,}?[)）]", "", text)
text = re.sub("(\n){2,}", "", text)
print("len(text): {}".format(len(text)))

chars = list(set(text))
char2id = {char:i for i,char in enumerate(chars)}
id2char = {v:k for k,v in char2id.items()}

FLAGS.num_labels = len(chars)

FLAGS.num_train_steps = int(
    len(text) / FLAGS.train_batch_size * FLAGS.num_train_epochs)
FLAGS.num_warmup_steps = int(FLAGS.num_train_steps * FLAGS.warmup_proportion)

print(FLAGS.num_train_steps, FLAGS.num_warmup_steps)

len(text): 4600600
1437687 143768


In [8]:

"""
data generator
"""

tokenizer = tokenization.FullTokenizer(vocab_file=FLAGS.vocab_file,
    do_lower_case=FLAGS.do_lower_case)

def data_generator(text, batch_size):

    while 1:
        batch_length = random.randint(FLAGS.batch_min_length, 
            FLAGS.batch_min_length)

        batch_input_ids = []
        batch_label_ids = []
        batch_input_mask = []
        batch_seg_ids = []

        for _ in range(batch_size):
            start_idx = random.randint(0, len(text)-batch_length-1)

            one_chars = text[start_idx : start_idx+batch_length]
            tokens = tokenizer.tokenize(one_chars)
            tokens = ['[CLS]'] + tokens +['[SEP]']
            input_ids = tokenizer.convert_tokens_to_ids(tokens)
            
            next_char = text[start_idx+batch_length]
            next_id = char2id[next_char]

            input_mask = [1] * len(input_ids)
            segment_ids = [0] * len(input_ids)


            # The mask has 1 for real tokens and 0 for padding tokens. Only real
            # tokens are attended to.
            while len(input_ids) < batch_length+2:
                input_ids.append(0)
                input_mask.append(0)
                segment_ids.append(0)

            assert len(input_ids)==len(input_mask)==len(segment_ids),(
                len(input_ids),len(input_mask),len(segment_ids))

            batch_input_ids.append(input_ids)
            batch_label_ids.append(next_id)
            batch_input_mask.append(input_mask)
            batch_seg_ids.append(segment_ids)

        feed_dict = {
            "input_ids_pl:0":np.stack(batch_input_ids),
            "label_ids_pl:0": np.stack(batch_label_ids),
            "input_mask_pl:0":np.stack(batch_input_mask),
            "segment_ids_pl:0":np.stack(batch_seg_ids)
        }

        yield feed_dict


In [9]:

"""
model class
"""
class GeneModel:
    """ class for build text generation model """
    
    def __init__(self):
        
        self.graph = tf.Graph()
        
        with self.graph.as_default():

            bert_config = modeling.BertConfig.from_json_file(FLAGS.bert_config_file)

            self.input_ids_pl = tf.placeholder(dtype=tf.int32, shape=[None, None], name='input_ids_pl')
            self.label_ids_pl = tf.placeholder(dtype=tf.int32, shape=[None,], name='label_ids_pl')
            self.input_mask_pl = tf.placeholder(dtype=tf.int32, shape=[None, None], name='input_mask_pl')
            self.segment_ids_pl = tf.placeholder(dtype=tf.int32, shape=[None, None], name='segment_ids_pl')

            model = modeling.BertModel(
                config=bert_config,
                is_training=FLAGS.is_training,
                input_ids=self.input_ids_pl,
                input_mask=self.input_mask_pl,
                token_type_ids=self.segment_ids_pl,
                use_one_hot_embeddings=False)
            
            self.saver_base = tf.train.Saver()

            output_layer = model.get_pooled_output()
            self.output_layer = output_layer

            hidden_size = output_layer.shape[-1].value

            output_weight = tf.get_variable(
              "output_weights", [FLAGS.num_labels, hidden_size],
              initializer=tf.truncated_normal_initializer(stddev=0.02)
            )
            output_bias = tf.get_variable(
              "output_bias", [FLAGS.num_labels], initializer=tf.zeros_initializer()
            )

            with tf.variable_scope("loss"):
                if FLAGS.is_training:
                    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)
                logits = tf.matmul(output_layer, output_weight, transpose_b=True)
                logits = tf.nn.bias_add(logits, output_bias)
                
            self.out = tf.nn.softmax(logits)    
            self.logits = logits

            loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels=self.label_ids_pl, logits=self.logits
            )
            self.loss = tf.reduce_mean(loss)

            self.train_op = optimization.create_optimizer(self.loss, FLAGS.learning_rate, 
                FLAGS.num_train_steps, FLAGS.num_warmup_steps, False)
            
            self.saver_all = tf.train.Saver()
            self.g_init = tf.global_variables_initializer()
    

geneModel = GeneModel()
print('model create done.')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
model create done.


In [10]:
"""
test model
"""

sess = tf.Session(graph=geneModel.graph)
sess.run(geneModel.g_init)
geneModel.saver_base.restore(sess, FLAGS.init_checkpoint)
print("model weights restored.")

train_gen = data_generator(text, 2)

for i in range(2):
    fetches = [geneModel.logits, geneModel.out, geneModel.loss, geneModel.train_op]
    feed_dict = next(train_gen)
    logits, out, loss, op = sess.run(fetches=fetches, feed_dict=feed_dict)
    print("logits:", logits)
    print("out:", out)
    print("loss:", loss)
    print("op:", op)

sess.close()

INFO:tensorflow:Restoring parameters from chinese_L-12_H-768_A-12/bert_model.ckpt
model weights restored.
logits: [[ 0.5032296  -0.2547218   0.37694877 ... -1.0822551  -0.5499307
   0.02438326]
 [ 0.8119055  -0.18089297  0.01411045 ... -1.0508754   0.03014636
  -0.03249807]]
out: [[3.17242288e-04 1.48667823e-04 2.79606989e-04 ... 6.49865979e-05
  1.10664776e-04 1.96530862e-04]
 [4.26764775e-04 1.58132694e-04 1.92181062e-04 ... 6.62510938e-05
  1.95287692e-04 1.83429293e-04]]
loss: 8.510294
op: None
logits: [[ 0.39249372 -0.1629214   0.31307676 ... -0.9772467  -0.43508345
   0.15311816]
 [ 0.09771284 -0.24254797  0.26303375 ... -0.99336195 -0.36380297
  -0.05494519]]
out: [[2.8262599e-04 1.6218041e-04 2.6104884e-04 ... 7.1835871e-05
  1.2353783e-04 2.2246034e-04]
 [2.1041710e-04 1.4972959e-04 2.4824415e-04 ... 7.0669703e-05
  1.3263167e-04 1.8062691e-04]]
loss: 8.812802
op: None


In [4]:
"""
predict
"""

def predict(start_str=None, batch_length=None, steps=100, sess=None):

    if sess is None:
        sess = tf.Session(graph=geneModel.graph)
        sess.run(geneModel.g_init)
        geneModel.saver_all.restore(sess, os.path.join(FLAGS.ckpt_weights, 'weights'))
        print("model weights restored.")
        close_sess = True
    else:
        close_sess = False

    if start_str is None and batch_length is None:
        batch_length = 20
    elif batch_length is None:
        batch_length = len(start_str)

    if start_str is None:
        start_idx = random.randint(0, len(text)-batch_length-1)
        one_chars = text[start_idx : start_idx+batch_length]
    else:
        one_chars = start_str

    print("original text: ", one_chars)

    generated_text = ""
    for i in range(steps):

        batch_input_ids = []
        #     batch_label_ids = []
        batch_input_mask = []
        batch_seg_ids = []
    
        tokens = tokenizer.tokenize(one_chars)
        tokens = ['[CLS]'] + tokens +['[SEP]']
        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        input_mask = [1] * len(input_ids)
        segment_ids = [0] * len(input_ids)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        while len(input_ids) < batch_length+2:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)

        assert len(input_ids)==len(input_mask)==len(segment_ids),(
            len(input_ids),len(input_mask),len(segment_ids))

        batch_input_ids.append(input_ids)
        batch_input_mask.append(input_mask)
        batch_seg_ids.append(segment_ids)

        feed_dict = {
            "input_ids_pl:0":np.stack(batch_input_ids),
            "input_mask_pl:0":np.stack(batch_input_mask),
            "segment_ids_pl:0":np.stack(batch_seg_ids)
        }

        out = sess.run(fetches=geneModel.out, feed_dict=feed_dict)[0]
        next_char = id2char[np.argmax(out)]
        generated_text += next_char

        one_chars += next_char
        one_chars = one_chars[1:]

    print("generated text: ", generated_text)

    if close_sess:
        sess.close()


In [20]:
predict()

INFO:tensorflow:Restoring parameters from drive/My Drive/gene_ckpt/weights
model weights restored.
original text:  身形一动。化为一团金光腾空而起。而叶楚此
generated text:  女则站在那里，一动不动。韩立眉头一皱，心里有些惊讶。不过这时，韩立却不知道，这是他在他身上的一种不知名的天雷竹，他对此物的价格也不知有多少。但是他现在的时间，实在不是一个月后的事情。韩立心中一沉，但随


In [21]:

""" train
"""

sess = tf.Session(graph=geneModel.graph)
sess.run(geneModel.g_init)
geneModel.saver_base.restore(sess, FLAGS.init_checkpoint)
print("init_checkpoint weights restored.")

train_gen = data_generator(text, FLAGS.train_batch_size)
log_str = "epoch: {}, batch: {} / {}, batch_train_loss: {}, time: {}"

for epoch in range(FLAGS.num_train_epochs):
    
    batch_train_loss_list = []
    batch_num = len(text) // FLAGS.train_batch_size
    start = time.time()
    for batch_i in range(batch_num):
        
        fetches = [geneModel.loss, geneModel.train_op]
        batch_train_loss,_ = sess.run(fetches=fetches, feed_dict=next(train_gen))
        batch_train_loss_list.append(batch_train_loss)
        
        if batch_i % 2000 == 0:
            print(log_str.format(
                epoch+1, (batch_i+1), batch_num, np.mean(batch_train_loss_list), time.time()-start
            ))
            start = time.time()
            predict(start_str="韩立自信的点点头，", sess=sess)
                
    os.makedirs(FLAGS.ckpt_weights, exist_ok=True)
    geneModel.saver_all.save(sess, os.path.join(FLAGS.ckpt_weights, 'weights'))

sess.close()
        

INFO:tensorflow:Restoring parameters from chinese_L-12_H-768_A-12/bert_model.ckpt
init_checkpoint weights restored.
epoch: 1, batch: 1 / 143768, batch_train_loss: 8.59052848815918, time: 4.574858903884888
original text:  韩立自信的点点头，
generated text:  缅交缅缅交伦缅缅揍交唇漱爵缅缅缅缅缅爵缅缅祗揍气谢缅缅缅交揍缅缅缅气交爵揍缅覃缅困缅缅揍缅缅交揍缅揍缅缅气气缅缅缅交缅危唇缅交交揍缅昧缅交缅交爵缅缅揍缅揍昧缅缅缅揍缅交交缅揍奢鸟缅缅缅揍交缅缅昧爵谢缅
epoch: 1, batch: 2001 / 143768, batch_train_loss: 8.333712577819824, time: 301.83274006843567
original text:  韩立自信的点点头，
generated text:  ，瓶子，危危，，子子子，危危，危子危危，子子，危，子，子危子子，，，子子，，危危子，子，子危子，子子交子危子唇交子子子子子子子，子瓶危，危子子，危危危危子子子子入子子，危子子子危子子子入子危，子子子子
epoch: 1, batch: 4001 / 143768, batch_train_loss: 7.923469066619873, time: 313.7700946331024
original text:  韩立自信的点点头，
generated text:  ，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，有，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，
epoch: 1, batch: 6001 / 143768, batch_train_loss: 7.514791965484619, time: 313.8668236732483
original text:  韩立自信的点点头，
generated text:  ，，，，的，。，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，的

epoch: 1, batch: 68001 / 143768, batch_train_loss: 4.732569217681885, time: 282.5918047428131
original text:  韩立自信的点点头，
generated text:  但是韩立却不禁一怔，但随后一怔，但随后就不再说什么了。“韩立一听此言，心中一怔，但随后就不再迟疑的说道：“这里是我们一个大厅，在一个大殿内，一个小山上，一个大小不一的小山，一个个小山的石峰，一个小山的
epoch: 1, batch: 70001 / 143768, batch_train_loss: 4.699703216552734, time: 282.7236678600311
original text:  韩立自信的点点头，
generated text:  但是韩立却没有露出了什么意外之色。“不错，这是什么东西？”韩立听了一笑，不禁一怔。“不错，这些东西，我们也不会再说的。”韩立听到这话一惊，心中一动，但是心中却一动。“不错，这里的灵气不少，但是在天渊城
epoch: 1, batch: 72001 / 143768, batch_train_loss: 4.667304039001465, time: 282.6863503456116
original text:  韩立自信的点点头，
generated text:  但是他也不知道，这些东西是何种物。”韩立一怔，但是脸上却露出一丝不知。“这是什么？”韩立一怔。“这是什么？”韩立一怔，但也没有说什么。“韩立一怔，但是脸上也露出一丝异样。“这是什么？”韩立一听此话，脸
epoch: 1, batch: 74001 / 143768, batch_train_loss: 4.636229991912842, time: 282.759135723114
original text:  韩立自信的点点头，
generated text:  就将玉简中的灵药，一个个都是一个个的小小小小的，但是一个个都是小小的小小小小的一个小小的小山，一个个都是一个个小小小的小小的小小的小小的小小的一个小小的小山，一个个都是一个个小小的小小小小的小小的一个
epoch: 1, batch: 76001 / 143768, batch_train_loss

generated text:  将目光一收，一闪的向前飞去。韩立在一座小山上坐着。“韩立一听此话，脸上一沉，不禁问道。“这是什么？”韩立神色一动，不动声色的说道。“这个，我们也不是普通的修士，而是一位结丹期修士，一见此幕，立刻一阵的
epoch: 1, batch: 138001 / 143768, batch_train_loss: 4.049606800079346, time: 282.7979462146759
original text:  韩立自信的点点头，
generated text:  然后一抬手，一个白色小瓶从手心处飞出，一个黑色小人从空中激射而出，一下将那些飞剑一下全都一下被一层黑色光罩，然后从中激出一团团的黑色雾气，从雾气中一下消失了。韩立一惊，但马上就想起了什么，脸色一下苍白
epoch: 1, batch: 140001 / 143768, batch_train_loss: 4.038786888122559, time: 282.84958004951477
original text:  韩立自信的点点头，
generated text:  就不客气的走了进去。“你们这个人，我们这个人也不会轻易动手的。”韩立不动声色的说道。“这个不错，这位道友是否真的如此说了？”韩立心中一动，但是这些人中的那名男子，正站在一座小山之上，但是在山峰的上空，
epoch: 1, batch: 142001 / 143768, batch_train_loss: 4.028271675109863, time: 282.8411626815796
original text:  韩立自信的点点头，
generated text:  就不再说什么，只是一副不想再说的样子。韩立不禁有些不知道。这一下，韩立也不敢怠慢，只是一副不敢再说的样子。“这个不可思议的事情，就是这一次的大战，让他有些不太好。这时，韩立已经进入了一个不小的小洞，就
epoch: 2, batch: 1 / 143768, batch_train_loss: 3.4654674530029297, time: 0.15775370597839355
original text:  韩立自信的点点头，
generated text:  不禁的问道。“这些人都是结丹期修士，不是一个不可思议的东

epoch: 2, batch: 62001 / 143768, batch_train_loss: 3.1663410663604736, time: 282.82866287231445
original text:  韩立自信的点点头，
generated text:  并没有说什么。“这个东西，我们可以一齐收下。”韩立一听此言，心里大喜，急忙将神识一扫，脸色一沉，脸上露出了一丝笑容。“这位道友，不用担心了。”韩立一笑的说道。“这个自然了。”韩立听了此言，心中一松。“
epoch: 2, batch: 64001 / 143768, batch_train_loss: 3.1635470390319824, time: 282.994562625885
original text:  韩立自信的点点头，
generated text:  就不再说什么了。”韩某也是一名元婴期修士，一见此幕，立刻大喜的一下爆裂开来。一声惊天动地的巨响传来。韩立一怔，但随后就想起了什么，不由得想到了什么。这一下，韩立心中一凛。这些东西，竟是一种灵药。韩立心
epoch: 2, batch: 66001 / 143768, batch_train_loss: 3.160907745361328, time: 282.8315677642822
original text:  韩立自信的点点头，
generated text:  并说道。“这些人是不是有些不好的。”韩立淡淡的说道。“这些东西，我们也是知道的。”韩立一笑，接着将手中的玉简一抛，然后往空中一挥手中的一只手臂一抖，一股浓浓浓的灵气滚滚之气，让韩立一怔。“这些东西不是
epoch: 2, batch: 68001 / 143768, batch_train_loss: 3.1587483882904053, time: 282.76007103919983
original text:  韩立自信的点点头，
generated text:  就不再理会的说道：“这些人也不是普通的修士，这些人也是一个个炼气期的修士，一个都是结丹期的修士，也不会轻易放过的。”韩立一听此言，脸上露出一丝笑容，但马上就恢复如常了。韩立心中一动，但马上就想要施展神
epoch: 2, batch: 70001 / 143768, batch_train

generated text:  就不再说什么了。韩立听到此话，脸上露出一丝笑容。“这些东西，还真是个不小的东西。”这是什么？”韩立一见此幕，心中一凛。这些人，竟然是那些人。这让韩立有些意外。但是他的神识一扫下，顿时一下现了那只巨大的
epoch: 2, batch: 132001 / 143768, batch_train_loss: 3.0894298553466797, time: 281.37029337882996
original text:  韩立自信的点点头，
generated text:  就将手中的玉简，放在了其中。“这是什么？”韩立一惊，急忙问道。“，你们的确是一名元婴期修士。此刻，韩立才放心的将神识放出了神念，看似无法看清楚其中的情形。这些人虽然身上都有一层护罩，但是这些法阵中的禁
epoch: 2, batch: 134001 / 143768, batch_train_loss: 3.0877883434295654, time: 281.8833725452423
original text:  韩立自信的点点头，
generated text:  就一声的离开了此地。他正在一旁的一块玉简中。“这是我的那个小子！”韩立听了此话，微微一笑。“这些东西，我可以在你身上一点印记住，我可以将你的小命交给你来了。”韩立一声低沉，但随即一声低沉的怪异的怪叫声
epoch: 2, batch: 136001 / 143768, batch_train_loss: 3.085681676864624, time: 281.60040974617004
original text:  韩立自信的点点头，
generated text:  就将玉佩一收进了储物袋中。“这是什么？”韩立有些奇怪的问道。“这是什么，这是什么？”韩立一听此言，脸上一丝讶色闪过。“这是什么东西？”韩立一见此物，脸色一变。“”“韩立一见此幕，脸上露出了一丝古怪之色
epoch: 2, batch: 138001 / 143768, batch_train_loss: 3.0839645862579346, time: 281.7513098716736
original text:  韩立自信的点点头，
generated text:  并不再说什么。“这次，这位师姐，可是一直在这

...

epoch: 4, batch: 50001 / 143768, batch_train_loss: 2.7476253509521484, time: 281.5410189628601
original text:  韩立自信的点点头，
generated text:  就带着其他人，向那个方向飞去。韩立神念一动，那些银丝就从袖口中飞射而出一个盘旋后就化为一道青虹的飞出了。“这人是我的记名弟子，我们也不会轻易的。”韩立神色一动的说道。“这个自然。我们也不会让这些人过于
epoch: 4, batch: 52001 / 143768, batch_train_loss: 2.7457406520843506, time: 281.4571032524109
original text:  韩立自信的点点头，
generated text:  也没有说什么。“这个自然。”韩立一笑的回道。“这是什么？”韩立一听此言，心中一动，但随即又有些模糊不清了。韩立见此，心中一动，但是他一听此言，脸色一下大变。“你是什么人？”韩立一听这话，心里一动，但是
epoch: 4, batch: 54001 / 143768, batch_train_loss: 2.7449872493743896, time: 281.6937413215637
original text:  韩立自信的点点头，
generated text:  就带着韩立，向西边的西边而去。这一次，韩立却一口气将这些噬金虫全都吞噬掉。这些噬金虫虽然不是普通的虫兽，但是这些妖兽的妖丹，还是炼制法宝的最佳的材料。这些东西，都是上古修士炼制的。”韩立脸色阴沉的说道
epoch: 4, batch: 56001 / 143768, batch_train_loss: 2.743730068206787, time: 281.6022238731384
original text:  韩立自信的点点头，
generated text:  就不再理会对方的话语，走到了韩立身前，并把手一抱肩，就走了过去。“韩道友，你这是什么意思？”韩立有些惊讶的问道。“这个自然。我们可以安全的出手了。”韩道友不会想在此地。”韩立神色如常地说道。“这个自然
epoch: 4, batch: 58001 / 143768, batch_train_

generated text:  但是表面神色如常，但心中一动，但也没有马上回头，只是一脸的不满。“这个自然。这些人是一些小辈的弟，怎会如此的不知情。”韩立听了这话，微微一笑后，就带着韩立离开了此地。韩立自然不会有什么不满之意。“这个
epoch: 4, batch: 120001 / 143768, batch_train_loss: 2.709338903427124, time: 281.2919373512268
original text:  韩立自信的点点头，
generated text:  并没有再说什么。“这个自然。不过，这些人的修为低下，根本无法看出什么的。”韩立神色一动，淡淡的说道：“这位道友，你这话是什么意思？”韩立一听此言，心中一动，当即将神念一扫，却现那名人影正站在一旁，正和
epoch: 4, batch: 122001 / 143768, batch_train_loss: 2.7081737518310547, time: 281.3176655769348
original text:  韩立自信的点点头，
generated text:  就带着韩立一齐飞到了此地。韩立目光一扫，就在这片地方。韩立目光一扫，就看到那只巨猿，正用神识扫过此兽，看看是否有什么厉害禁制，韩立还不知道。但韩立却不知道，在这片地方，不但是一座巨大山峰，却是天南第一
epoch: 4, batch: 124001 / 143768, batch_train_loss: 2.7070558071136475, time: 281.17673993110657
original text:  韩立自信的点点头，
generated text:  就开始了炼制法宝的事情。这让韩立有些意外。“这是一种叫叫“天地元”的。”韩立冷静的说道。“这就好。我们也不用担心的。”韩立一笑，不在意的说道。“这个自然不会。我们还是先将那些妖王的事情告诉我，我会在你
epoch: 4, batch: 126001 / 143768, batch_train_loss: 2.7061235904693604, time: 281.38478350639343
original text:  韩立自信的点点头，
generated text:  就带着韩立离开了此地。韩立也不知道，他在这种

epoch: 5, batch: 44001 / 143768, batch_train_loss: 2.6067113876342773, time: 282.7133343219757
original text:  韩立自信的点点头，
generated text:  就将玉简收了起。“这是？”韩立一见此人，不禁大喜。“这是？”韩立一见此幕脸色一沉身形一晃身形一晃人形傀儡就诡异的出现在了韩立身前，，一张口，一团青光喷出，一只手掌一翻，一个玉盒出现在了手中。“这是一种
epoch: 5, batch: 46001 / 143768, batch_train_loss: 2.60560941696167, time: 284.3472807407379
original text:  韩立自信的点点头，
generated text:  就将手中的玉简收了起来。“这就是天晶真人，你们也是第一次见到，但是是一名身材高大，面目普通，但是一身青色长袍，一身青色战甲，一身黑袍的样子。这些人一见韩立身形，一立，正好落在了那只巨猿身上。这时，巨猿
epoch: 5, batch: 48001 / 143768, batch_train_loss: 2.6042332649230957, time: 282.9828555583954
original text:  韩立自信的点点头，
generated text:  就将神识抽出，将整个人都收进了储物袋中。“这是？”韩立一怔，但随即神色一动，单手一抬，一道金弧射出，一下击在了巨大光球上。巨大光球表面灵光一闪，一道青白色遁光激射而来。“轰”的一声巨响，一道金弧从虚空
epoch: 5, batch: 50001 / 143768, batch_train_loss: 2.6026248931884766, time: 283.14413356781006
original text:  韩立自信的点点头，
generated text:  然后手一扬，一团青光从手中飞射而出，一个盘旋后，化为一只数尺大的小鸟，双翅一展下，一下没入虚空中不见了踪影。韩立见此，心中一凛，不加思索的一张口，一团蓝色冰焰喷出，正好将此女包裹在了其中。此女一见韩立
epoch: 5, batch: 52001 / 143768, batch_train_

generated text:  但随即又沉声道：“你们也走吧。我们也走吧。”韩立一笑的说道。“这个自然。我们先去看看那些东西。”韩立目光在那些石屋上一扫后，就将石屋重新收起，再次闭上双目起来。这一次，韩立并没有动用什么，反而口中念念
epoch: 5, batch: 114001 / 143768, batch_train_loss: 2.572051525115967, time: 282.6435704231262
original text:  韩立自信的点点头，
generated text:  就不再迟疑的走了进去。一进入此地，韩立就不再迟疑的走了过去。“韩兄，你也到了。我们已经走了。”韩立神色一动，淡淡的说道：“我们几个，不要动了。”韩立淡淡的说道。“这个自然。我们快动手。”一名黑袍大汉面
epoch: 5, batch: 116001 / 143768, batch_train_loss: 2.5711491107940674, time: 282.8849604129791
original text:  韩立自信的点点头，
generated text:  就化为一道青虹，飞进了一个不大的小山谷。此山谷不算大，但是山谷内的灵气却丝毫不减，仿佛真的被那些黑色光柱击的粉碎，一下化为了无形。韩立见此，心中一动，但还是不敢过去。“韩立望了一眼远处的黑色山峰，正在
epoch: 5, batch: 118001 / 143768, batch_train_loss: 2.5703227519989014, time: 282.5983204841614
original text:  韩立自信的点点头，
generated text:  就不再多说什么了。“这位道友，你也来了。”韩立目光一扫，不动声色的走进石门中。在门外，正有一名白袍女子和一名身穿白袍的中年人，身穿一件白色长袍，但是一闪后，就此消失的无了。韩立见此，心中一喜。他可不想
epoch: 5, batch: 120001 / 143768, batch_train_loss: 2.569370746612549, time: 282.5970175266266
original text:  韩立自信的点点头，
generated text:  就带着曲魂离开了此地。韩立心中一凛，急忙将神识放出

KeyboardInterrupt: 